In [41]:
from matplotlib import pyplot as plt
import cv2
import subprocess as sub
import numpy as np
import ruptures as rpt
import os
#import classify_images.py

#img = cv2.imread('./frames/output_0023.png')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#plt.imshow(img)
#plt.title('a picture')
#plt.show()

In [50]:
print('Creating frames')
p = sub.Popen(['ffmpeg', '-i', 'Calgary Modern Contemporary Home Video Tour  - 522 37th St SW-MxIg2ZO8ftg.f137.mp4', '-r', '2', './frames2/output_%04d.png'],stdout=sub.PIPE,stderr=sub.PIPE)
output, errors = p.communicate()
print('Done creating frames')


b''
b"ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers\n  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609\n  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtw

In [52]:
print('Building vectors and classifying images')
p = sub.Popen(['python', 'classify_images.py', 'frames2/*'],stdout=sub.PIPE,stderr=sub.PIPE)
output, errors = p.communicate()
#print(output)
#print(errors)
print('Done')
#classify_images.py "frames/*"

b"parsing 0 frames2/output_0159.png \n\nresults for frames2/output_0159.png\nsliding door (score = 0.57180)\n\n\nresults for frames2/output_0159.png\npatio, terrace (score = 0.14396)\n\n\nresults for frames2/output_0159.png\nmobile home, manufactured home (score = 0.09619)\n\n\nresults for frames2/output_0159.png\npicket fence, paling (score = 0.01314)\n\n\nresults for frames2/output_0159.png\nprison, prison house (score = 0.01142)\n\n\nparsing 1 frames2/output_0206.png \n\nresults for frames2/output_0206.png\nwashbasin, handbasin, washbowl, lavabo, wash-hand basin (score = 0.79201)\n\n\nresults for frames2/output_0206.png\nrestaurant, eating house, eating place, eatery (score = 0.04254)\n\n\nresults for frames2/output_0206.png\nbathtub, bathing tub, bath, tub (score = 0.02300)\n\n\nresults for frames2/output_0206.png\ntub, vat (score = 0.02146)\n\n\nresults for frames2/output_0206.png\ndesk (score = 0.01715)\n\n\nparsing 2 frames2/output_0195.png \n\nresults for frames2/output_0195.pn

In [ ]:
import json

%pylab inline
pylab.rcParams['figure.figsize'] = (14, 6)

print('Loading vectors')
arr = np.array([ np.loadtxt('./image_vectors/' + 'output_0001.png.npz') ])

for f in os.listdir('./image_vectors')[1:]:
  arr = np.concatenate(( arr, [np.loadtxt('./image_vectors/' + f)]), axis=0)

print('Predicting change points')
model = 'l2'
#algo = rpt.Dynp(model = model, min_size=1, jump=1)
algo = rpt.Window(width=2, model=model)#.fit(signal)
bks = algo.fit_predict(arr, n_bkps = 43)
print('Done')
print('Change points: ')
print(bks)

#rpt.display(arr, bks)

if bks[0] != 0:
    bks.insert(0, 1)

bks_pairs = zip(bks[:-1], bks[1:])

with open('./image_to_labels.json') as f:
    labels = json.load(f)
    
for pair in bks_pairs:
    fig = plt.figure()
    #print(pair)
    for i in range(pair[0], pair[1]):
        num = pair[1] - pair[0]
        img = cv2.imread('./frames2/output_0%03d.png' % i)
        ax1 = fig.add_subplot(1,num,i-pair[0]+1)
        ax1.set_title('pic %03d' % i)
        ax1.imshow(img)
    #for i in range(pair[0], pair[1]):
     #   ax1.text(1,1, labels['frames2/output_0%03d.png' % i])
    fig.show()
    
# print(bk)